In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw
from sklearn.utils import class_weight
from datetime import datetime

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
df_train = pd.read_csv(r'H:/3_output_raMSIn/df_train24.csv').set_index('pixel_id')
df_train.shape
df_ext = pd.read_csv(r'H:/3_output_raMSIn/df_ext24.csv').set_index('pixel_id')
df_ext.shape
df_FNA = pd.read_csv(r'H:/3_output_raMSIn/df_FNA24.csv').set_index('pixel_id')
df_FNA.shape

(88701, 19)

# 1. Data Pre-Processing

In [3]:
df_train = df_train.reset_index()
df_ext = df_ext.reset_index()
df_FNA = df_FNA.reset_index()

In [4]:
df_train

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.000000,782.762573,651.139648,855.941162,1289.565430,675.470398,578.552246,556.107544,0.00000,0.000000,0.000000,0.000000,1236.293091,1765.094360,904.144226,1229.976685,0.000000,485.318390,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,601.592896,713.357544,1621.182007,616.704224,728.034058,0.000000,974.713318,794.743958,0.00000,572.712952,510.829651,0.000000,2961.001221,2892.591553,986.521362,1167.839844,0.000000,921.005554,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.000000,0.000000,700.977356,1092.700562,684.122437,1090.359131,926.827942,1464.706055,0.00000,0.000000,0.000000,1168.753052,1675.238892,2225.103027,1134.286865,1621.029907,0.000000,0.000000,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.000000,627.533691,547.317627,993.356018,1254.330078,0.000000,889.944214,1968.154053,0.00000,0.000000,0.000000,768.893005,958.072998,2827.721436,1136.862183,2258.572754,0.000000,486.305145,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.000000,0.000000,813.711731,0.000000,968.478027,0.000000,732.884155,0.000000,0.00000,0.000000,0.000000,0.000000,2059.021484,1611.982788,609.699585,1659.533447,627.203186,571.359680,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,2450.656738,3595.426758,950.979004,0.000000,0.000000,1016.526367,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.000000,0.000000,0.000000,1108.048462,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,2368.329102,2122.539551,1904.507690,0.000000,0.000000,0.000000,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.000000,0.000000,1333.430664,1270.490479,0.000000,0.000000,1763.664062,950.907288,2025.98584,0.000000,1544.935181,0.000000,1841.424072,2093.726318,1692.966187,0.000000,0.000000,0.000000,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.000000,0.000000,0.000000,1174.360474,0.000000,0.000000,0.000000,0.000000,1196.84436,0.000000,1019.172302,0.000000,2354.068604,1721.714355,1216.751465,0.000000,1628.065918,0.000000,1


In [5]:
df_train["Sample"] = 0
df_train["Row"] = 0
df_train["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_train["pixel_id"])):
    Sample.append("_".join(str(df_train["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_train["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_train["pixel_id"][i]).split("_")[-2]))
df_train["Sample"] = Sample
df_train["Row"] = Row
df_train["Scan"] = Scan

In [6]:
df_ext["Sample"] = 0
df_ext["Row"] = 0
df_ext["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_ext["pixel_id"])):
    Sample.append("_".join(str(df_ext["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_ext["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_ext["pixel_id"][i]).split("_")[-2]))
df_ext["Sample"] = Sample
df_ext["Row"] = Row
df_ext["Scan"] = Scan

In [7]:
df_FNA["Sample"] = 0
df_FNA["Row"] = 0
df_FNA["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(df_FNA["pixel_id"])):
    Sample.append("_".join(str(df_FNA["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(df_FNA["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(df_FNA["pixel_id"][i]).split("_")[-2]))
df_FNA["Sample"] = Sample
df_FNA["Row"] = Row
df_FNA["Scan"] = Scan

In [8]:
pop_column = df_train.pop('Scan')
df_train.insert(1, 'Scan', pop_column)
pop_column = df_train.pop('Row')
df_train.insert(1, 'Row', pop_column)
pop_column = df_train.pop('Sample')
df_train.insert(1, 'Sample', pop_column)

pop_column = df_ext.pop('Scan')
df_ext.insert(1, 'Scan', pop_column)
pop_column = df_ext.pop('Row')
df_ext.insert(1, 'Row', pop_column)
pop_column = df_ext.pop('Sample')
df_ext.insert(1, 'Sample', pop_column)

pop_column = df_FNA.pop('Scan')
df_FNA.insert(1, 'Scan', pop_column)
pop_column = df_FNA.pop('Row')
df_FNA.insert(1, 'Row', pop_column)
pop_column = df_FNA.pop('Sample')
df_FNA.insert(1, 'Sample', pop_column)

In [9]:
df_train = df_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ext = df_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_FNA = df_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [10]:
raMSIn = pd.concat([df_train, df_ext, df_FNA])

In [11]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ = raMSIn.loc[raMSIn["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [12]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.0,545.338989,0.000000,498.940277,0.0,0.0,...,0.0,0.0,0.000000,8060.281738,3152.019287,0.000000,1218.151733,0.000000,801.547180,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,0.0,0.000000,677.591492,2300.148682,0.0,0.0,...,0.0,0.0,0.000000,9737.291992,3041.586182,0.000000,1063.171875,0.000000,1082.132202,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,0.0,0.000000,880.095154,1930.286377,0.0,0.0,...,0.0,0.0,0.000000,11845.030273,3117.169434,0.000000,1317.922852,0.000000,1191.340332,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,0.0,0.000000,8586.939453,0.000000,0.0,0.0,...,0.0,0.0,0.000000,10544.400391,6283.457520,0.000000,0.000000,0.000000,1537.444458,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,0.0,0.000000,7384.484375,0.000000,0.0,0.0,...,0.0,0.0,0.000000,10030.952148,7803.814941,0.000000,0.000000,0.000000,1983.716309,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,0.0,691.663818,966.017944,0.000000,0.0,0.0,...,0.0,0.0,0.000000,920.265137,1684.391724,687.658691,0.000000,0.000000,0.000000,0
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,0.0,0.000000,943.075989,0.000000,0.0,0.0,...,0.0,0.0,597.175964,811.593140,1974.694214,783.007935,676.766235,0.000000,0.000000,0
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,0.0,0.000000,854.914978,0.000000,0.0,0.0,...,0.0,0.0,0.000000,822.435730,2245.246094,860.293518,491.932281,499.042358,686.713989,0
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,0.0,0.000000,5242.995117,850.380005,0.0,0.0,...,0.0,0.0,0.000000,2198.789795,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [13]:
HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,68,0.0,2133.882812,0.000000,10731.034180,1000.316101,0.000000,...,0.0,911.586731,1431.151367,7287.519043,8025.071289,1179.324585,0.000000,0.0,3265.985840,1
1,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,69,0.0,3771.562256,0.000000,11126.564453,0.000000,0.000000,...,0.0,0.000000,1669.991211,7418.794922,8084.173340,0.000000,0.000000,0.0,4774.750488,1
2,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,70,0.0,1480.239868,0.000000,10383.758789,0.000000,0.000000,...,0.0,0.000000,967.886353,6008.706055,7036.358887,0.000000,0.000000,0.0,3356.994873,1
3,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,71,0.0,1731.181641,0.000000,12172.352539,0.000000,0.000000,...,0.0,1407.395142,1993.890991,8018.412598,7888.710938,1153.661865,0.000000,0.0,3840.479248,1
4,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,2,110,0.0,2336.908691,1136.738892,11670.751953,0.000000,0.000000,...,0.0,0.000000,2124.743896,7886.218262,5014.947754,0.000000,0.000000,0.0,3419.675781,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8831,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,159,0.0,3107.359131,0.000000,7271.594238,1800.728882,0.000000,...,0.0,0.000000,2475.537598,2858.967773,5764.550293,0.000000,0.000000,0.0,3824.137207,1
8832,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,160,0.0,3283.487061,1027.628174,6070.109375,1265.771362,0.000000,...,0.0,0.000000,1463.356079,2338.229492,5076.562500,1430.687378,0.000000,0.0,4052.958496,1
8833,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,161,0.0,3806.802734,0.000000,7600.413086,1265.341553,0.000000,...,0.0,0.000000,3323.055420,2398.373047,6157.842773,1745.956909,1649.523438,0.0,2525.834473,1
8834,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1F...,HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW,94,162,0.0,4041.713623,0.000000,7891.657227,1201.307129,1531.936646,...,0.0,1666.665649,3783.354492,2882.657715,5609.024414,1244.879761,0.000000,0.0,5148.016602,1


In [14]:
sample_list_ = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_]
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"]

# 2. Visualization

In [120]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)


In [121]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/whole/"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)


In [15]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/ROI/"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)


In [16]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [17]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Greens")

In [18]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


# 4. Generating TIF Preview for the Top 18

In [21]:
def genAROItif_UwU(sample_ROI, sample_, name, mz, vmin_, vmax_):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    #mean         0.196373
    #std          0.180211
    #min          0.000000
    #25%          0.116014
    #50%          0.173645
    #75%          0.229128
    #max          2.152269
    #plt.imshow(df_HM, cmap=cmap_w2a, vmin=0.229128/0.5, vmax=2.152269*0.25, zorder=1)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=vmin_, vmax=vmax_, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [22]:
mzfeatures = ["869.7241", "909.5496", "269.2486", "736.6464", "535.4734", 
              "559.4732", "609.5114", "682.5922", "846.5858", "633.5108", 
              "445.3172", "683.5948", "269.0878", "215.0328", "283.255", 
              "671.4661", "610.5128", "175.0246"]

In [23]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/raMSI_inFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2554)

In [24]:
raMSI_ML = df_raMSI_ML.reset_index()

In [25]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [26]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [27]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [28]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [29]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [32]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

# = 0
for TOP in list(mzfeatures):
    
    #vmax_ = vmax_list[v]
    counter = 0
    #start = datetime.now()
    #for df in ROI_sample_list:
    for df in sample_list:
        start = datetime.now()
        
        name = name_list[counter]
        sample_ = sample_list[counter]
        preview = sample_list_[counter][[TOP]]
        for i in preview.columns:
            preview[i] = preview[i].astype(float)
        vmax_list = list(preview.describe().iloc[-1])
        #vmin_list = list(preview.describe().iloc[-3])
        #vmax_list = list(preview.describe().iloc[-2])
        #vmin_ = vmin_list[0] * 0.25
        vmin_ = 0
        vmax_ = vmax_list[0]*0.8
        #print(TOP, name, vmax_)
        
        genAROItif_UwU(sample_ROI=sample_list_[counter], sample_=sample_, name=name, mz=TOP, vmin_=vmin_, vmax_=vmax_)
        #print(df.shape)
        counter += 1
        end = datetime.now()
        Duration = end - start
        print(Duration)


0:00:27.211385
0:00:30.117041
0:00:34.088601
0:00:18.322859
0:00:21.520787
0:00:29.565761
0:00:01.883900
0:00:01.358840
0:00:02.070869
0:00:01.994899
0:01:06.008108
0:01:04.653317
0:01:09.766657
0:00:30.852145
0:00:25.948970
0:00:22.077774
0:00:21.891379
0:00:21.074096
0:00:21.536661
0:00:01.586690
0:00:03.167536
0:00:01.754545
0:00:01.597894
0:00:58.825177
0:00:59.789981
0:01:06.845685
0:00:25.715344
0:00:28.790687
0:00:33.421424
0:00:18.076382
0:00:21.214112
0:00:29.072367
0:00:01.879685
0:00:01.321533
0:00:02.046850
0:00:01.985879
0:01:02.755256
0:01:00.356544
0:01:05.281561
0:00:30.018836
0:00:25.393634
0:00:21.325362
0:00:21.176802
0:00:20.688820
0:00:21.054286
0:00:01.589133
0:00:03.147289
0:00:01.755884
0:00:01.646406
0:00:58.646278
0:00:59.561027
0:01:06.680757
0:00:25.791452
0:00:28.733891
0:00:33.789303
0:00:18.199008
0:00:22.032560
0:00:30.109925
0:00:01.923807
0:00:01.390348
0:00:02.154660
0:00:02.018247
0:01:04.627775
0:01:02.384074
0:01:09.762332
0:00:34.327120
0:00:47.23

# 5. TIF Stacking

In [33]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
        
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/merge"
if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)
    

In [85]:
for folders in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI"):
    for filename in os.listdir(folders):
        sample = ("_").join(filename.split("\\")[-1].split("_")[:-1])
        #print(sample)
        if sample == "ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2":
            try:
                source = os.path.join(folders, filename)
                target = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\ROI_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"
                target = os.path.join(target, filename)
                shutil.move(source, target)

            except:
                print("done")
                pass

In [111]:
import glob
import tifffile

with tifffile.TiffWriter(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\ROIstack_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4.tif") as stack:
    for filename in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\ROI_HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4\*.tif"):
        stack.save(
            tifffile.imread(filename), 
            contiguous=True
        )

# 6. TIF Merging (Not now)

In [112]:
def TIFMerger(TIFStack, TIFMerge):
    ret, images = cv2.imreadmulti(TIFStack, [], cv2.IMREAD_UNCHANGED)
    alp = 0.9
    img1 = images[0]
    img2 = images[1]
    added_image = cv2.addWeighted(img1, alp, img2, alp, 0)
    for i in range(2, 18):
        img = images[i]
        added_image = cv2.addWeighted(added_image, alp, img, alp**i, 0)
        cv2.imwrite(TIFMerge, added_image)

In [144]:
TIFMerger(
    TIFStack= r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\ROIstack_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2.tif", 
    TIFMerge= r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\merge\ROImerge_HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2.tif"
)

# 6. U-Net

## 6.1 N-Stack Samples

In [26]:
new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/stack/0_HKULiver_Post29wk_HCCAMCLiver"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)

new_imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/ROI/stack/1_HKULiver_Post29wk_HCCLiver"

if not os.path.exists(new_imagesavepath):
    os.makedirs(new_imagesavepath)


In [26]:
images = []

dict_labels = {"0_HKULiver_Post29wk_HCCAMCLiver":0, "1_HKULiver_Post29wk_HCCLiver":1}

#dir_labels = os.listdir(glob.glob("img_noLT_TICN_TIF_db/output/stack/label"))
size = (512,512)

for folders in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\*_HKULiver_*"):
    print(folders, "reading images...")
    
    for filename in os.listdir(folders):
        label = folders.split("\\")[-1]
        print(os.path.join(folders, filename))
        try:
            ret, img = cv2.imreadmulti(os.path.join(folders, filename), [], cv2.IMREAD_UNCHANGED)
            
            if img is not None:
                in_img_list = []

                for i in range(len(img)):
                    candidate = img[i]
                    in_img = 255 - candidate[:, :, 3]
                    in_img = cv2.resize(in_img, dsize=size)
                    in_img_list.append(in_img)
                images.append(in_img_list)
                #print(len(images))

                
        except:
            print(os.path.join(folders, filename), "cannot read image")
            pass

H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver reading images...
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver\ROIstack_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver\ROIstack_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver\ROIstack_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver\ROIstack_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULiver_Post29wk_HCCAMCLiver\ROIstack_HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\stack\0_HKULi

In [27]:
len(images[0][0][0])  # 26,18,64,64

512

## 6.2 N-Stack labels

In [28]:
labels = []

for GTfolders in glob.glob(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label"):
    print(GTfolders, "reading images...")
    
    for filename in os.listdir(GTfolders):
        #label = GTfolders.split("\\")[-1]
        print(os.path.join(GTfolders, filename))
        try:
            ret, img = cv2.imreadmulti(os.path.join(GTfolders, filename), [], cv2.IMREAD_UNCHANGED)
            
            if img is not None:
                label_img_list = []

                for i in range(len(img)):
                    candidate = img[i]
                    label_img = 255 - candidate[:, :, 3]
                    label_img = cv2.resize(label_img, dsize=size)
                    label_img_list.append(label_img)
                labels.append(label_img_list)
                #print(len(images))
                #labels.append(dict_labels[label])
                
        except:
            print(os.path.join(GTfolders, filename), "cannot read image")
            pass

H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label reading images...
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2.tif
H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\ROI\label\ROImerge_HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4.tif
H:\3_ou

In [29]:
len(labels[0][0][0])  # 26,1,64,64

512

## 6.3 NPA Rearragement for Samples

In [30]:
def rearrangement(im):
    
    re_depth = []  # for each depth
    re_row = []  # for each row
    re_scan = []  # for each scan
    re_inte = []

    for d in range(len(im)):  # len(im) = 1824
        for y in range(len(im[d])):  # len = 572
            for x in range(len(im[d])):  # len = 572
                
                re_inte.append(im[d][y][x])
                re_scan.append(x)
                re_row.append(y)
                re_depth.append(d)
        
    df_im = pd.DataFrame()
    df_im['re_inte'] = re_inte
    df_im['re_scan'] = re_scan
    df_im['re_row'] = re_row
    df_im['re_depth'] = re_depth

    return df_im

In [31]:
def NParray_generator(im):
    import warnings
    warnings.filterwarnings('ignore')
    
    df_im = rearrangement(im)
    df_im = df_im.sort_values(["re_row", "re_scan", "re_depth"], ascending = [True, True, True]).reset_index().drop(columns = ["index", "re_row", "re_scan", "re_depth"])
    df_im = df_im.T
    arr = df_im.values.copy()
    arr.resize(262144, 18)
    
    df_image = pd.DataFrame(arr)
    NPA_im = []
    for i in range(512):
        df_row = df_image.iloc[(0+i*512):(512+i*512)]
        arr_row = df_row.to_numpy(dtype=np.uint8)
        NPA_im.append(arr_row)
    arr_image = np.array(NPA_im)
    return arr_image

In [32]:
NPA_images = []
for i in range(len(images)):
    print(f"Rearranging image {i+1}...")
    arr_image = NParray_generator(im = images[i])
    NPA_images.append(arr_image)
    print(f"done for arr_image {i+1}")

Rearranging image 1...
done for arr_image 1
Rearranging image 2...
done for arr_image 2
Rearranging image 3...
done for arr_image 3
Rearranging image 4...
done for arr_image 4
Rearranging image 5...
done for arr_image 5
Rearranging image 6...
done for arr_image 6
Rearranging image 7...
done for arr_image 7
Rearranging image 8...
done for arr_image 8
Rearranging image 9...
done for arr_image 9
Rearranging image 10...
done for arr_image 10
Rearranging image 11...
done for arr_image 11
Rearranging image 12...
done for arr_image 12
Rearranging image 13...
done for arr_image 13
Rearranging image 14...
done for arr_image 14
Rearranging image 15...
done for arr_image 15
Rearranging image 16...
done for arr_image 16
Rearranging image 17...
done for arr_image 17
Rearranging image 18...
done for arr_image 18
Rearranging image 19...
done for arr_image 19
Rearranging image 20...
done for arr_image 20
Rearranging image 21...
done for arr_image 21
Rearranging image 22...
done for arr_image 22
Rearra

In [33]:
len(NPA_images[0][0][0])  # 26,64,64,18

18

## 6.4 NPA Rearragement for Labels

In [34]:
def NParray_generator_label(im):
    import warnings
    warnings.filterwarnings('ignore')
    
    df_im = rearrangement(im)
    df_im = df_im.sort_values(["re_row", "re_scan", "re_depth"], ascending = [True, True, True]).reset_index().drop(columns = ["index", "re_row", "re_scan", "re_depth"])
    df_im = df_im.T
    arr = df_im.values.copy()
    arr.resize(262144, 1)
    
    df_image = pd.DataFrame(arr)
    NPA_im = []
    for i in range(512):
        df_row = df_image.iloc[0+i*512:512+i*512]
        arr_row = df_row.to_numpy(dtype=np.uint8)
        NPA_im.append(arr_row)
    arr_image = np.array(NPA_im)
    return arr_image

In [35]:
def rearrangement_label(im):
    
    re_depth = []  # for each depth
    re_row = []  # for each row
    re_scan = []  # for each scan
    re_inte = []

    for y in range(len(im)):  # len = 128
        for x in range(len(im[y])):  # len = 128

            re_inte.append(im[y][x])
            re_scan.append(x)
            re_row.append(y)
            re_depth.append(1)
        
    df_im = pd.DataFrame()
    df_im['re_inte'] = re_inte
    df_im['re_scan'] = re_scan
    df_im['re_row'] = re_row
    df_im['re_depth'] = re_depth

    return df_im

In [36]:
NPA_labels = []
for l in range(len(labels)):
    print(f"Rearranging label image {l+1}...")
    arr_labelimage = NParray_generator_label(im = labels[l])
    NPA_labels.append(arr_labelimage)
    print(f"done for arr_labelimage {l+1}")

Rearranging label image 1...
done for arr_labelimage 1
Rearranging label image 2...
done for arr_labelimage 2
Rearranging label image 3...
done for arr_labelimage 3
Rearranging label image 4...
done for arr_labelimage 4
Rearranging label image 5...
done for arr_labelimage 5
Rearranging label image 6...
done for arr_labelimage 6
Rearranging label image 7...
done for arr_labelimage 7
Rearranging label image 8...
done for arr_labelimage 8
Rearranging label image 9...
done for arr_labelimage 9
Rearranging label image 10...
done for arr_labelimage 10
Rearranging label image 11...
done for arr_labelimage 11
Rearranging label image 12...
done for arr_labelimage 12
Rearranging label image 13...
done for arr_labelimage 13
Rearranging label image 14...
done for arr_labelimage 14
Rearranging label image 15...
done for arr_labelimage 15
Rearranging label image 16...
done for arr_labelimage 16
Rearranging label image 17...
done for arr_labelimage 17
Rearranging label image 18...
done for arr_labeli

In [37]:
len(NPA_labels[0][0][0])  # 26,64,64,1

1

In [38]:
GT_labels = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1], dtype=np.int32)

In [39]:
print(len(NPA_images[0]), len(NPA_labels[0]))

512 512


In [72]:
# defining a function for Scenario A images previewing
def show_images_new_prediction(images, predictions, start_id, num=26):
    
    plt.gcf().set_size_inches(12,14)
    
    if num>26: num=26
    for i in range(0, num):
        ax=plt.subplot(6,5, 1+i)
        ax.imshow(images[start_id])
        if (len(predictions) > 0):
            #title = "ai = " 
            #title += "\nlabel = " + str(predictions[start_id])
            title = "\nlabel = " + str(predictions[start_id])
        else:
            title = "label = "
        ax.set_title(title, fontsize=12)
        ax.set_xticks([]); ax.set_yticks([])
        start_id += 1
    plt.set_cmap("Greens_r")
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/previewBackgroundSamples_2D.jpg", dpi=300)
    plt.close()

In [69]:
# calling the defined function for saving the previewed images of Scenario A
show_images_new_prediction(NPA_labels, GT_labels, 0)

## 6.5 Preparation of training and test dataset

In [40]:
print(len(NPA_images), len(NPA_labels), len(GT_labels))

26 26 26


In [41]:
UwU_train_feature = np.concatenate([NPA_images[3:13], NPA_images[16:]])
UwU_test_feature = np.concatenate([NPA_images[0:3], NPA_images[13:16]])
UwU_train_label = np.concatenate([NPA_labels[3:13], NPA_labels[16:]])
UwU_test_label = np.concatenate([NPA_labels[0:3], NPA_labels[13:16]])
UwU_train_GTlabel = np.append(GT_labels[3:13], GT_labels[16:])
UwU_test_GTlabel = np.append(GT_labels[0:3], GT_labels[13:16])

In [42]:
UwU_train_feature = np.array(UwU_train_feature, dtype=np.float32)
UwU_test_feature = np.array(UwU_test_feature, dtype=np.float32)
UwU_train_label = np.array(UwU_train_label, dtype=np.float32)
UwU_test_label = np.array(UwU_test_label, dtype=np.float32)

In [43]:
print(len(UwU_train_feature), len(UwU_test_feature))
print(UwU_train_feature.shape, UwU_train_label.shape)
print(UwU_test_feature.shape, UwU_test_label.shape)

20 6
(20, 512, 512, 18) (20, 512, 512, 1)
(6, 512, 512, 18) (6, 512, 512, 1)


In [44]:
imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/"
if not os.path.exists(imagesavepath):
    os.makedirs(imagesavepath)
    
np.save(imagesavepath + "UwU_train_feature.npy", UwU_train_feature)
print("UwU_train_feature.npy is saved.")
np.save(imagesavepath + "UwU_test_feature.npy", UwU_test_feature)
print("UwU_test_feature.npy is saved.")
np.save(imagesavepath + "UwU_train_label.npy", UwU_train_label)
print("UwU_train_label.npy is saved.")
np.save(imagesavepath + "UwU_test_label.npy", UwU_test_label)
print("UwU_test_label.npy is saved.")

UwU_train_feature.npy is saved.
UwU_test_feature.npy is saved.
UwU_train_label.npy is saved.
UwU_test_label.npy is saved.


imagesavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/"
if not os.path.exists(imagesavepath):
    os.makedirs(imagesavepath)

UwU_train_feature = np.load(imagesavepath + "UwU_train_feature.npy")
UwU_test_feature = np.load(imagesavepath + "UwU_test_feature.npy")
UwU_train_label = np.load(imagesavepath + "UwU_train_label.npy")
UwU_test_label = np.load(imagesavepath + "UwU_test_label.npy")


In [45]:
UwU_train_feature_vector = UwU_train_feature.reshape(len(UwU_train_feature), 512,512,18).astype("float32")
UwU_test_feature_vector = UwU_test_feature.reshape(len(UwU_test_feature), 512,512,18).astype("float32")

In [46]:
UwU_train_feature_normalize = UwU_train_feature_vector/255
UwU_test_feature_normalize = UwU_test_feature_vector/255

In [47]:
UwU_train_label_vector = UwU_train_label.reshape(len(UwU_train_label), 512,512,1).astype("float32")
UwU_test_label_vector = UwU_test_label.reshape(len(UwU_test_label), 512,512,1).astype("float32")

In [48]:
UwU_train_label_normalize = UwU_train_label_vector/255
UwU_test_label_normalize = UwU_test_label_vector/255

In [49]:
# converting lables into the fashion of one-hot encoding
train_label_2Donehot = np_utils.to_categorical(UwU_train_GTlabel)
test_label_2Donehot = np_utils.to_categorical(UwU_test_GTlabel)

UwU_train_GTlabel = np.expand_dims(UwU_train_GTlabel, axis=-1)

UwU_test_GTlabel = np.expand_dims(UwU_test_GTlabel, axis=-1) 

## 6.6 Model Preparation

In [50]:
model = models.load_model(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/UwU_raMSIn.h5")

try:
    model.load_weights(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/UwU_raMSIn.weight")
    print("import CNN model Completed Successfully!")
except:
    print("import CNN model Failed!")

OSError: No file or directory found at H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/UwU_raMSIn.h5

In [51]:
del model

NameError: name 'model' is not defined

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]


def unet_model(input_shape, output_channels):
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    #chanDim = -1
    
    # 1st layer 1st CONV => RELU layer set (64)
    L1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
    L1 = tf.keras.layers.Dropout(0.1)(L1)
    # 1st layer 2nd CONV => RELU => POOL layer set
    L1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L1)
    P1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L1)
    
    # 2nd layer 1st CONV => RELU layer set (32)
    L2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P1)
    L2 = tf.keras.layers.Dropout(0.1)(L2)
    # 2nd layer 2nd CONV => RELU => POOL layer set
    L2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L2)
    P2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L2)
    
    # 3rd layer 1st CONV => RELU layer set (16)
    L3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P2)
    L3 = tf.keras.layers.Dropout(0.2)(L3)
    # 3rd layer 2nd CONV => RELU => POOL layer set
    L3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L3)
    P3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L3)
    
    # 4th layer 1st CONV => RELU layer set (8)
    L4 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P3)
    L4 = tf.keras.layers.Dropout(0.2)(L4)
    # 4th layer 2nd CONV => RELU => POOL layer set
    L4 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L4)
    P4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L4)
    
    L5 = tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P4)
    L5 = tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L5)

    # ===TRANSPOSE===
    U6 = tf.keras.layers.UpSampling2D((2, 2))(L5)
    U6 = tf.keras.layers.concatenate([U6, L4])
    L6 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', kernel_initializer="he_normal", padding='same')(U6)
    L6 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', kernel_initializer="he_normal", padding='same')(L6)
    
    # 4th layer 1st CONV => RELU layer set (16)
    U7 = tf.keras.layers.UpSampling2D((2, 2))(L6)
    U7 = tf.keras.layers.concatenate([U7, L3])
    L7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U7)
    L7 = tf.keras.layers.Dropout(0.2)(L7)
    # 4th layer 2nd CONV => RELU => POOL layer set
    L7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L7)
    
    U8 = tf.keras.layers.UpSampling2D((2, 2))(L7)
    U8 = tf.keras.layers.concatenate([U8, L2])
    # 3rd layer 1st CONV => RELU layer set (32)
    L8 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U8)
    L8 = tf.keras.layers.Dropout(0.2)(L8)
    # 3rd layer 2nd CONV => RELU => POOL layer set
    L8 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L8)
    #model.add(BatchNormalization(axis=chanDim))
    
    U9 = tf.keras.layers.UpSampling2D((2, 2))(L8)
    U9 = tf.keras.layers.concatenate([U9, L1])
    # 2nd layer 1st CONV => RELU layer set (64)
    L9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U9)
    L9 = tf.keras.layers.Dropout(0.1)(L9)
    # 2nd layer 2nd CONV => RELU => POOL layer set
    L9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L9)
    
    outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='sigmoid')(L9)
    
    return tf.keras.models.Model(inputs=[inputs], outputs=[outputs])

# Define the nested U-Net
def nested_unet(input_shape, output_channels):
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # First U-Net
    first_unet = unet_model(input_shape, 1)  # Output a single channel
    first_unet_output = first_unet(inputs)
    
    # Second U-Net is applied to the output of the first U-Net
    second_unet = unet_model((64, 64, 1), output_channels)  # Accept the output of the first U-Net
    second_unet_output  = second_unet(first_unet_output )

    # Final output
    #final_outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='sigmoid' if output_channels == 1 else 'softmax')(second_unet_output )
    
    return tf.keras.models.Model(inputs=[inputs], outputs=[second_unet_output])

tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)

# Example usage
input_shape  = (64, 64, 18)  # Hyperspectral image with 18 channels
output_channels = 1  # Change to 3 for 3-channel output
#code_size= 16

#model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model = nested_unet(input_shape, output_channels)

# initialize the learning rate choices and optimizer
lr = 1e-5
opt = Adam(learning_rate=lr)

# compile the model
#model.compile(optimizer=opt, loss="categorical_crossentropy", binary_crossentropy
#              metrics=METRICS)
model.compile(optimizer=opt, loss='binary_crossentropy' if output_channels == 1 else 'categorical_crossentropy', 
              metrics=METRICS)


In [52]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]


def unet_model(input_shape, output_channels):
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    #chanDim = -1
    
    # 1st layer 1st CONV => RELU layer set (64)
    L1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
    #L1 = tf.keras.layers.Dropout(0.1)(L1)
    # 1st layer 2nd CONV => RELU => POOL layer set
    L1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L1)
    P1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L1)
    
    # 2nd layer 1st CONV => RELU layer set (32)
    L2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P1)
    #L2 = tf.keras.layers.Dropout(0.1)(L2)
    # 2nd layer 2nd CONV => RELU => POOL layer set
    L2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L2)
    P2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L2)
    
    # 3rd layer 1st CONV => RELU layer set (16)
    L3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P2)
    L3 = tf.keras.layers.Dropout(0.1)(L3)
    # 3rd layer 2nd CONV => RELU => POOL layer set
    L3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L3)
    P3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L3)
    
    # 4th layer 1st CONV => RELU layer set (8)
    L4 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P3)
    L4 = tf.keras.layers.Dropout(0.1)(L4)
    # 4th layer 2nd CONV => RELU => POOL layer set
    L4 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L4)
    P4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(L4)
    
    L5 = tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(P4)
    L5 = tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L5)

    # ===TRANSPOSE===
    U6 = tf.keras.layers.UpSampling2D((2, 2))(L5)
    U6 = tf.keras.layers.concatenate([U6, L4])
    L6 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', kernel_initializer="he_normal", padding='same')(U6)
    L6 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', kernel_initializer="he_normal", padding='same')(L6)
    
    # 4th layer 1st CONV => RELU layer set (16)
    U7 = tf.keras.layers.UpSampling2D((2, 2))(L6)
    U7 = tf.keras.layers.concatenate([U7, L3])
    L7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U7)
    L7 = tf.keras.layers.Dropout(0.2)(L7)
    # 4th layer 2nd CONV => RELU => POOL layer set
    L7 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L7)
    
    U8 = tf.keras.layers.UpSampling2D((2, 2))(L7)
    U8 = tf.keras.layers.concatenate([U8, L2])
    # 3rd layer 1st CONV => RELU layer set (32)
    L8 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U8)
    L8 = tf.keras.layers.Dropout(0.2)(L8)
    # 3rd layer 2nd CONV => RELU => POOL layer set
    L8 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L8)
    #model.add(BatchNormalization(axis=chanDim))
    
    U9 = tf.keras.layers.UpSampling2D((2, 2))(L8)
    U9 = tf.keras.layers.concatenate([U9, L1])
    # 2nd layer 1st CONV => RELU layer set (64)
    L9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(U9)
    #L9 = tf.keras.layers.Dropout(0.1)(L9)
    # 2nd layer 2nd CONV => RELU => POOL layer set
    L9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_normal", padding="same")(L9)
    
    outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='sigmoid')(L9)
    
    return tf.keras.models.Model(inputs=[inputs], outputs=[outputs])

# Define the nested U-Net
def nested_unet(input_shape, output_channels):
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # First U-Net
    first_unet = unet_model(input_shape, 1)  # Output a single channel
    first_unet_output = first_unet(inputs)
    
    # Second U-Net is applied to the output of the first U-Net
    second_unet = unet_model((512, 512, 1), output_channels)  # Accept the output of the first U-Net
    second_unet_output  = second_unet(first_unet_output )
    F1 = tf.keras.layers.Flatten()(second_unet_output) #(P3)
    D1 = tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer="he_normal")(F1) #128
    D1 = tf.keras.layers.Dropout(0.5)(D1)  #0.2
    label_output = tf.keras.layers.Dense(units=2, activation="sigmoid")(D1)  #(L6) sigmoid
    
    # Final output
    #final_outputs = tf.keras.layers.Conv2D(output_channels, (1, 1), activation='sigmoid' if output_channels == 1 else 'softmax')(second_unet_output )
    
    return tf.keras.models.Model(inputs=[inputs], outputs=[label_output])

tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)

# Example usage
input_shape  = (512, 512, 18)  # Hyperspectral image with 18 channels
output_channels = 1  # Change to 3 for 3-channel output
#code_size= 16

#model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model = nested_unet(input_shape, output_channels)

# initialize the learning rate choices and optimizer
lr = 1e-3
opt = Adam(learning_rate=lr)

# compile the model
#model.compile(optimizer=opt, loss="categorical_crossentropy", binary_crossentropy
#              metrics=METRICS)
model.compile(optimizer=opt, loss='binary_crossentropy' if output_channels == 1 else 'categorical_crossentropy', 
              metrics=METRICS)


In [53]:
# define the path to our output directory
OUTPUT_PATH = imagesavepath

# initialize the input shape and number of classes
INPUT_SHAPE = (512, 512, 18)
NUM_CLASSES = 2

# define the total number of epochs to train, batch size, and the
# early stopping patience
EPOCHS = 5000
BATCH_SIZE = 1
EARLY_STOPPING_PATIENCE = 3  #3

labelNames = ["0_HKULiver_Post29wk_HCCAMCLiver", "1_HKULiver_Post29wk_HCCLiver"]

# initialize an early stopping callback to prevent the model from
# overfitting/spending too much time training with minimal gains
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=EARLY_STOPPING_PATIENCE,
    restore_best_weights=True)


In [54]:
# computing class weight based on appearance of each class
cls_wgts = class_weight.compute_class_weight(class_weight="balanced", 
                                             classes=np.unique(UwU_train_GTlabel), 
                                             y=UwU_train_GTlabel)
# dict mapping
cls_wgts = {i : cls_wgts[i] for i, label in enumerate(np.unique(UwU_train_GTlabel))}

## 6.7 Model Training

In [55]:
# recording the duration of CNN hyperparameters tuning
start = datetime.now()

# building the best model for training
print("[INFO] training the best model...")
EPOCHS = 5000
BATCH_SIZE = 1
EARLY_STOPPING_PATIENCE = 3
print("EPOCHS = ", EPOCHS , ", BS = ", BATCH_SIZE , ", ESP = ", EARLY_STOPPING_PATIENCE)

History_imbalanced = model.fit(x=UwU_train_feature_normalize, y=train_label_2Donehot, 
                               validation_data=(UwU_test_feature_normalize, test_label_2Donehot), 
                               batch_size=BATCH_SIZE, class_weight=cls_wgts, 
                               epochs=EPOCHS, callbacks =[early_stopping], verbose=1)

end = datetime.now()
Duration = end - start
print(Duration) 

[INFO] training the best model...
EPOCHS =  5000 , BS =  1 , ESP =  3
Epoch 1/5000
20/20 [==============================] - 508s 25s/step - loss: 0.9814 - tp: 9.0000 - fp: 11.0000 - tn: 9.0000 - fn: 11.0000 - accuracy: 0.4500 - precision: 0.4500 - recall: 0.4500 - auc: 0.3913 - prc: 0.4390 - val_loss: 0.6932 - val_tp: 3.0000 - val_fp: 3.0000 - val_tn: 3.0000 - val_fn: 3.0000 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 0.5000 - val_auc: 0.5000 - val_prc: 0.5000
Epoch 2/5000
20/20 [==============================] - 504s 25s/step - loss: 0.6935 - tp: 10.0000 - fp: 10.0000 - tn: 10.0000 - fn: 10.0000 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.5000 - auc: 0.4513 - prc: 0.4549 - val_loss: 0.6932 - val_tp: 3.0000 - val_fp: 3.0000 - val_tn: 3.0000 - val_fn: 3.0000 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 0.5000 - val_auc: 0.5000 - val_prc: 0.5000
Epoch 3/5000
20/20 [==============================] - 498s 25s/step - loss: 0.6933 - tp: 10.0000 - fp: 1

In [58]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 18)]    0         
                                                                 
 model (Functional)          (None, 512, 512, 1)       31387585  
                                                                 
 model_1 (Functional)        (None, 512, 512, 1)       31377793  
                                                                 
 flatten (Flatten)           (None, 262144)            0         
                                                                 
 dense (Dense)               (None, 64)                16777280  
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130 

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
import numpy as np
import matplotlib.pyplot as plt

In [63]:
model.save(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/UwU_raMSIn.h5")
print("UwU_raMSIn.h5 Saving Completed!")
model.save_weights(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/UwU_raMSIn.weight")
print("UwU_raMSI.weight Model Parameters Saving Completed!")

UwU_raMSIn.h5 Saving Completed!
UwU_raMSI.weight Model Parameters Saving Completed!


successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

x = NPA_images[0]
z = []
z_row = []
z.append(x)
z=np.array(z, dtype=np.float32)

print(x.shape)
print(z.shape)

z = z/255.0

successive_feature_maps = visualization_model.predict(z)
layer_names = [layer.name for layer in model.layers]

mapsavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/signal20map/"
if not os.path.exists(mapsavepath):
    os.makedirs(mapsavepath)

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    print(feature_map.shape)
    if len(feature_map.shape) == 4:
    
    # Plot Feature maps for the conv / maxpool layers, not the fully-connected layers
        n_features = feature_map.shape[-1]  # number of features in the feature map
        size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
        # We will tile our images in this matrix
        display_grid = np.zeros((size, size * n_features))
        #print(display_grid, size)

        # Postprocess the feature to be visually palatable
        for i in range(n_features):
            z  = feature_map[0, :, :, i]
            z -= z.mean()
            z /= z.std ()
            z *=  64
            z += 128
            z  = np.clip(z, 0, 255).astype('uint8')
            #print(z)
            # Tile each filter into a horizontal grid
            display_grid[:, i * size : (i + 1) * size] = z
    # Display the grid
        scale = 20. / n_features
        plt.figure( figsize=(scale * n_features, scale) )
        plt.title ( layer_name )
        plt.grid  ( False )
        plt.imshow( display_grid, aspect='auto', cmap='Greens' )
        plt.rcParams['figure.figsize'] = [10, 10]
        
        plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/signal20map/signal20map_{layer_name}.jpg", dpi=300)

In [60]:
# Prepare the visualization model
inner_unet_layers = [layer for layer in model.layers if 'model' in layer.name]
inner_unet_model = tf.keras.models.Model(inputs = [layer.output for layer in inner_unet_layers[0].layers if 'input' in layer.name], 
                                         outputs = [layer.output for layer in inner_unet_layers[0].layers if 'conv' in layer.name])

# Prepare the input image
x = NPA_images[0]
z = np.expand_dims(x, axis=0)  # Add batch dimension
z = np.array(z, dtype=np.float32) / 255.0  # Normalize the input

print("Input shape:", x.shape)
print("Normalized input shape:", z.shape)

# Get the feature maps
successive_feature_maps = inner_unet_model.predict(z)
layer_names = [layer.name for layer in inner_unet_layers[1].layers if 'conv' in layer.name]

# Create directory for output maps
mapsavepath = r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/signal20map/"
if not os.path.exists(mapsavepath):
    os.makedirs(mapsavepath)

# Loop through the feature maps and save them
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    print(f"Feature map shape for {layer_name}: {feature_map.shape}")
    
    if len(feature_map.shape) == 4:  # Check for convolutional feature maps
        
        n_features = feature_map.shape[-1]  # Number of features in the feature map
        size = feature_map.shape[1]  # Feature map size (size, size)
        
        # Prepare the display grid
        display_grid = np.zeros((size, size * n_features))
        #print(display_grid, size)

        # Postprocess and tile feature maps
        for i in range(n_features):
            z  = feature_map[0, :, :, i]
            z -= z.mean()  # Normalize
            z /= z.std() + 1e-5  # Avoid division by zero
            z *=  64
            z += 128
            z  = np.clip(z, 0, 255).astype('uint8')
            display_grid[:, i * size:(i + 1) * size] = z
            
        # Display the grid
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='Greens')
        plt.axis('off')  # Hide axis
        plt.rcParams['figure.figsize'] = [10, 10]
        
        plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4FNA/CNN/output/signal20map/signal20map_{layer_name}.jpg", dpi=300)
        #plt.close()  # Close the figure to free memory

Input shape: (512, 512, 18)
Normalized input shape: (1, 512, 512, 18)
1/1 [==============================] - 3s 3s/step
Feature map shape for conv2d_19: (1, 512, 512, 64)
Feature map shape for conv2d_20: (1, 512, 512, 64)
Feature map shape for conv2d_21: (1, 256, 256, 128)
Feature map shape for conv2d_22: (1, 256, 256, 128)
Feature map shape for conv2d_23: (1, 128, 128, 256)
Feature map shape for conv2d_24: (1, 128, 128, 256)
Feature map shape for conv2d_25: (1, 64, 64, 512)
Feature map shape for conv2d_26: (1, 64, 64, 512)
Feature map shape for conv2d_27: (1, 32, 32, 1024)
Feature map shape for conv2d_28: (1, 32, 32, 1024)
Feature map shape for conv2d_29: (1, 64, 64, 512)
Feature map shape for conv2d_30: (1, 64, 64, 512)
Feature map shape for conv2d_31: (1, 128, 128, 256)
Feature map shape for conv2d_32: (1, 128, 128, 256)
Feature map shape for conv2d_33: (1, 256, 256, 128)
Feature map shape for conv2d_34: (1, 256, 256, 128)
Feature map shape for conv2d_35: (1, 512, 512, 64)
Feature m

In [61]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [65]:
def plot_metrics_2Dimbalanced(history=History_imbalanced):
    from matplotlib.pyplot import figure
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    metrics = ['loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric], 
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.legend('', frameon=False)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()
        plt.rcParams['figure.figsize'] = [10, 10]
        #figure(figsize=(8, 6))
        plt.savefig(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4FNA\CNN\output\metrics_2DTrial.jpg", dpi=300)
    plt.close()

In [66]:
plot_metrics_2Dimbalanced(history=History_imbalanced)

In [67]:
def plot_cm_2Dimbalanced(labels, predictions, p=0.5):
    cm_2Dimbalanced = confusion_matrix(labels, predictions > p)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm_2Dimbalanced, annot=True, fmt="d")
    plt.title('Imbalanced Confusion Matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.savefig("UwU_output\cm_2DTrial.jpg", dpi=600)
    plt.close()

    print('Legitimate Transactions Detected (True Negatives): ', cm_2Dimbalanced[0][0])
    print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm_2Dimbalanced[0][1])
    print('Fraudulent Transactions Missed (False Negatives): ', cm_2Dimbalanced[1][0])
    print('Fraudulent Transactions Detected (True Positives): ', cm_2Dimbalanced[1][1])
    print('Total Fraudulent Transactions: ', np.sum(cm_2Dimbalanced[1]))

In [70]:
model_results_2Dimbalanced = model.evaluate(UwU_test_feature_normalize, test_label_2Donehot, 
                                          batch_size=BATCH_SIZE, verbose=1)
for name, value in zip(model.metrics_names, model_results_2Dimbalanced):
    print(name, ': ', value)

6/6 [==============================] - 36s 6s/step - loss: 0.6932 - tp: 3.0000 - fp: 3.0000 - tn: 3.0000 - fn: 3.0000 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.5000 - auc: 0.5000 - prc: 0.5000             
loss :  0.6931501030921936
tp :  3.0
fp :  3.0
tn :  3.0
fn :  3.0
accuracy :  0.5
precision :  0.5
recall :  0.5
auc :  0.5
prc :  0.5


In [73]:
test_label_2Donehot

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [49]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [55]:
del model

In [56]:
model = models.load_model("UwU_output/UwU_raMSI.h5")

try:
    model.load_weights("UwU_output/UwU_raMSI.weight")
    print("import CNN model Completed Successfully!")
except:
    print("import CNN model Failed!")

import CNN model Completed Successfully!


In [57]:
preds_train = model.predict(UwU_train_feature_normalize, verbose=1)
preds_test = model.predict(UwU_test_feature_normalize, verbose=1)

1/1 [==============================] - 0s 497ms/step


In [155]:
maker = preds_test[2]

In [156]:
def rearrangement_regen(im):
    arr = im*255
    regen_depth = []  # for each depth
    regen_row = []  # for each row
    regen_scan = []  # for each scan
    regen_inte = []
    #print(len(arr))
    for y in range(len(arr)):  # len = 128
        #print(len(arr[d]))
        for x in range(len(arr[y])):  # len = 128
            #print(len(arr[d][y]))
            for d in range(len(arr[y][x])):  # len = 128
                #print((arr[d][y][x]))
                regen_inte.append(arr[y][x][d])
                regen_scan.append(x)
                regen_row.append(y)
                regen_depth.append(d)
    df_regen = pd.DataFrame()
    df_regen['regen_inte'] = regen_inte
    df_regen['regen_scan'] = regen_scan
    df_regen['regen_row'] = regen_row
    df_regen['regen_depth'] = regen_depth
    df_regen = df_regen.sort_values(["regen_depth", "regen_row", "regen_scan"], ascending = [True, True, True]).reset_index().drop(columns = ["index"])
    df_regened = pd.DataFrame()
    df_regened["Row"] = df_regen.loc[df_regen["regen_depth"] == 0]["regen_row"]
    df_regened["Scan"] = df_regen.loc[df_regen["regen_depth"] == 0]["regen_scan"]
    df_regened["Feature1"] = df_regen.loc[df_regen["regen_depth"] == 0]["regen_inte"]
    #df_regened["Feature2"] = df_regen.loc[df_regen["regen_depth"] == 1].reset_index().drop(columns = ["index"])["regen_inte"]
   
    return df_regened

In [157]:
df_regened = rearrangement_regen(im=maker)

In [158]:
df_regened["Feature1"] = np.uint8(df_regened["Feature1"])
#df_regened["Feature2"] = np.uint8(df_regened["Feature2"])


In [159]:
df_regened

,Row,Scan,Feature1
0,0,0,249
1,0,1,254
2,0,2,253
3,0,3,243
4,0,4,249
...,...,...,...
4091,63,59,251
4092,63,60,253
4093,63,61,249
4094,63,62,237


In [160]:
def regenall_TIFstack(sample_, name):

    for mz in list(sample_.columns[2:]):

        df_heatmap = sample_[["Row", "Scan", mz]]
        df_heatmap2 = df_heatmap.iloc[[0]]
        df_HM = df_heatmap.loc[df_heatmap["Row"] == 0].loc[df_heatmap["Scan"] == 0]
        Heat_Row = df_heatmap["Row"]
        Heat_Scan = df_heatmap["Scan"]

        for y in range(len(set(Heat_Row))):
            for x in range(1, len(set(Heat_Scan))+1):

                df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
                if len(df_in) == 1:
                    df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                    df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                    #hi += 1


                elif len(df_in) == 0:
                    df_heatmap2["Row"] = y+1
                    df_heatmap2["Scan"] = x
                    df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                    df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                    #hi += 1


        #print(name, mz, hi, len(df_HM))
        if "index" in df_HM.columns:
            df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
        else:
            df_HM = df_HM.drop(columns = ["Row", "Scan"])

        df_HM = df_HM.T
        New_y = []
        for y in range(1, len(set(Heat_Row))+1):
            New_y.append(str(y))
        New_x = []
        for x in range(1, len(set(Heat_Scan))+1):
            New_x.append(str(x))

        arr = df_HM.values.copy()
        arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

        df_HM = pd.DataFrame(arr)
        df_HM.index = New_y
        df_HM.columns = New_x

        for i in list(df_HM.columns):
            df_HM[i] = df_HM[i].astype(float)

        plt.subplots(dpi = 300)
        plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=255, zorder=1)
        plt.xticks([])
        plt.yticks([])
        plt.box(False)
        plt.autoscale(enable=True, tight=True)

        plt.rcParams['figure.figsize'] = [2, 2]

        plt.savefig(f"UwU_output/UwU_raMSI_ReGen_TIFstack_3.tif", bbox_inches = 'tight', pad_inches = 0)

In [161]:
import warnings
warnings.filterwarnings('ignore')

regenall_TIFstack(sample_=df_regened[["Row","Scan","Feature1"]], name="df_regened_F1")

In [236]:
del model

NameError: name 'model' is not defined